In [1]:
import mysql.connector

In [2]:
with open('password.txt', 'r') as f:
    password = f.read()

подключаемся

In [4]:
con0 = mysql.connector.connect(host='127.0.0.1', port=3306, database='youtube', user='root', password=password)

In [5]:
cur0=con0.cursor(dictionary=True)

проверяем что всё работает

In [7]:
cur0.execute("SELECT * FROM channels")

In [8]:
result = cur0.fetchall()

In [9]:
result

[{'id_channels': 1,
  'channel_name': '- Петькино -',
  'subscribers': 187000,
  'topic_id': 2,
  'blogger_id_channels': 1,
  'adv_a': Decimal('0'),
  'views': 293590,
  'pickle': 'petkino'},
 {'id_channels': 2,
  'channel_name': 'BARAKuda',
  'subscribers': 508000,
  'topic_id': 1,
  'blogger_id_channels': 2,
  'adv_a': Decimal('0'),
  'views': 797560,
  'pickle': 'barakuda'},
 {'id_channels': 3,
  'channel_name': 'varlamov',
  'subscribers': 196000000,
  'topic_id': 2,
  'blogger_id_channels': 3,
  'adv_a': Decimal('0'),
  'views': 307720000,
  'pickle': 'varlamov'}]

# CRUD

In [10]:
def my_str(x):
    if isinstance(x, str):
        x = "'" + x + "'"
    else:
        x = str(x)
    return x

In [11]:
class MyCur:
    def __init__(self, con):
        self.con = con
        self.cur = self.con.cursor(dictionary=True)
    
    def execute(self, sql_command):
        print(sql_command)
        try:
            self.cur.execute(sql_command)
            print('Команда выполнена успешно')
        except Exception as error:
            print(error)
            
    def create(self, table, columns=None, values=None):
        if columns != None:
            columns = ' (' + ', '.join(columns) + ')'
        else:
            columns = ''
        str_values = []
        for value in values:
            str_values.append(my_str(value))
        values = ', '.join(str_values)
        sql_command = "INSERT INTO {table}{columns} VALUES ({values})".format(table=table, columns=columns, values=values)
        self.execute(sql_command)
        self.con.commit()
        
        
    def read(self, table, columns='*', where=None):
        columns = ', '.join(columns)
        if where != None:
            where = ' WHERE ' + where
        else:
            where = ''
        sql_command = "SELECT {columns} FROM {table}{where}".format(table=table, columns=columns, where=where)
        self.execute(sql_command)
        data = self.cur.fetchall()
        return data
    
    def update(self, table, columns, values, where):
        pairs = []
        for i in range(len(columns)):
            column = columns[i]
            value = my_str(values[i])
            pairs.append(column + ' = ' + value)
        pairs = ', '.join(pairs)
        if where != None:
            where = ' WHERE ' + where
        sql_command = "UPDATE {table} SET {pairs}{where}".format(table=table, pairs=pairs, where=where)
        self.execute(sql_command)
        self.con.commit()
        
    def delete(self, table, where=None):
        if where != None:
            where = ' WHERE ' + where
        else:
            where = ''
        sql_command = "DELETE FROM {table}{where}".format(table=table, where=where)
        self.execute(sql_command)
        self.con.commit()

In [12]:
cur = MyCur(con0)

In [13]:
#посмотрим как выглядит таблица topics

In [14]:
print(cur.read('topics'))

SELECT * FROM topics
Команда выполнена успешно
[{'id_topics': 1, 'topic_name': 'comedy', 'conversion': Decimal('0.010')}, {'id_topics': 2, 'topic_name': 'travel', 'conversion': Decimal('0.050')}]


In [15]:
# и как называются темы, у которых конверсия больше 0,2

In [16]:
print(cur.read('topics', columns=['topic_name'], where="conversion > 0.02"))

SELECT topic_name FROM topics WHERE conversion > 0.02
Команда выполнена успешно
[{'topic_name': 'travel'}]


In [17]:
#удалим строку comedy

In [18]:
cur.delete('topics', where="topic_name = 'comedy'")

DELETE FROM topics WHERE topic_name = 'comedy'
Команда выполнена успешно


In [19]:
print(cur.read('topics'))

SELECT * FROM topics
Команда выполнена успешно
[{'id_topics': 2, 'topic_name': 'travel', 'conversion': Decimal('0.050')}]


In [20]:
# вернём назад строку comedy и добавим строку other

In [21]:
cur.create('topics', values = [1, 'comedy', 0.01])
cur.create('topics', values = [3, 'other', 0.007])

INSERT INTO topics VALUES (1, 'comedy', 0.01)
Команда выполнена успешно
INSERT INTO topics VALUES (3, 'other', 0.007)
Команда выполнена успешно


In [22]:
print(cur.read('topics'))

SELECT * FROM topics
Команда выполнена успешно
[{'id_topics': 1, 'topic_name': 'comedy', 'conversion': Decimal('0.010')}, {'id_topics': 2, 'topic_name': 'travel', 'conversion': Decimal('0.050')}, {'id_topics': 3, 'topic_name': 'other', 'conversion': Decimal('0.007')}]


In [23]:
# и заменим маленькую конверсию на нулевую, а названия тем с маленькой конверсией на zero

In [24]:
cur.update('topics', ['topic_name', 'conversion'], ['zero', 0], "conversion < 0.011")

UPDATE topics SET topic_name = 'zero', conversion = 0 WHERE conversion < 0.011
Команда выполнена успешно


In [25]:
print(cur.read('topics'))

SELECT * FROM topics
Команда выполнена успешно
[{'id_topics': 1, 'topic_name': 'zero', 'conversion': Decimal('0.000')}, {'id_topics': 2, 'topic_name': 'travel', 'conversion': Decimal('0.050')}, {'id_topics': 3, 'topic_name': 'zero', 'conversion': Decimal('0.000')}]


In [26]:
# вернём всё как было

In [27]:
cur.delete('topics')
cur.create('topics', values = [1, 'comedy', 0.01])
cur.create('topics', values = [2, 'travel', 0.05])

DELETE FROM topics
Команда выполнена успешно
INSERT INTO topics VALUES (1, 'comedy', 0.01)
Команда выполнена успешно
INSERT INTO topics VALUES (2, 'travel', 0.05)
Команда выполнена успешно


In [28]:
# добавим в остальные таблицы по несколько записей

In [29]:
import datetime

import pandas as pd

In [30]:
videos = pd.read_excel('videos.xlsx')

In [31]:
#videos['date'] = videos['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
#videos['date'] = videos['date'].apply(lambda x: datetime.datetime.fromtimestamp(x))
#videos['date'] = videos['date'].apply(lambda x: datetime.datetime.date(x))
videos['date'] = videos['date'].apply(str)
videos['adv_a'] = videos['adv_a'].apply(bool)

In [32]:
videos

,id,name,channel,views,date,adv_a,channel_id,blogger_id
0,1,Заветный берег на озере Иткуль,1,1530,2020-10-15 00:00:00,False,1,1
1,2,Путешествие в прошлое для МексЭтноЭксп,1,7751,2019-05-20 00:00:00,False,1,1
2,3,Бакшевская масляница 2019,1,12571,2019-03-15 00:00:00,False,1,1
3,4,Виталий Наливкин ловит наркоманов,2,1284187,2020-12-13 00:00:00,False,2,2
4,5,Виталий Наливкин рушит бетонные плиты,2,1707191,2020-11-22 00:00:00,True,2,2
5,6,Виталий Наливкин Масочный режим,2,1442579,2020-11-15 00:00:00,True,2,2
6,7,Виталий Наливкин спасает Камчатку,2,1101153,2020-10-25 00:00:00,True,2,2
7,8,Хорн: тихая голландская идиллия,3,152453,2019-08-05 00:00:00,False,3,3
8,9,ОМОН и москвичи: задержания в центре столицы,3,1270230,2019-08-04 00:00:00,True,3,3
9,10,"За что боролся, на то и напоролся: случай на м...",3,1920210,2019-08-03 00:00:00,False,3,3


In [33]:
cur.delete('videos')

DELETE FROM videos
Команда выполнена успешно


In [34]:
videos.apply(lambda x: cur.create('videos', values=list(x)), axis=1)

INSERT INTO videos VALUES (1, 'Заветный берег на озере Иткуль', 1, 1530, '2020-10-15 00:00:00', False, 1, 1)
Команда выполнена успешно
INSERT INTO videos VALUES (2, 'Путешествие в прошлое для МексЭтноЭксп', 1, 7751, '2019-05-20 00:00:00', False, 1, 1)
Команда выполнена успешно
INSERT INTO videos VALUES (3, 'Бакшевская масляница 2019', 1, 12571, '2019-03-15 00:00:00', False, 1, 1)
Команда выполнена успешно
INSERT INTO videos VALUES (4, 'Виталий Наливкин ловит наркоманов', 2, 1284187, '2020-12-13 00:00:00', False, 2, 2)
Команда выполнена успешно
INSERT INTO videos VALUES (5, 'Виталий Наливкин рушит бетонные плиты', 2, 1707191, '2020-11-22 00:00:00', True, 2, 2)
Команда выполнена успешно
INSERT INTO videos VALUES (6, 'Виталий Наливкин Масочный режим', 2, 1442579, '2020-11-15 00:00:00', True, 2, 2)
Команда выполнена успешно
INSERT INTO videos VALUES (7, 'Виталий Наливкин спасает Камчатку', 2, 1101153, '2020-10-25 00:00:00', True, 2, 2)
Команда выполнена успешно
INSERT INTO videos VALUES (8

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
dtype: object

In [35]:
channels = pd.read_excel('channels.xlsx')
channels

,id_channels,name,subscribers,category,blogger,adv_a,views,pickle
0,1,- Петькино -,187000,2,1,0.000000,293590,petkino
1,2,BARAKuda,508000,1,2,0.145833,797560,barakuda
2,3,varlamov,196000000,2,3,0.009524,307720000,varlamov


In [36]:
cur.delete('channels')

DELETE FROM channels
Команда выполнена успешно


In [37]:
channels.apply(lambda x: cur.create('channels', values = list(x)), axis=1)


INSERT INTO channels VALUES (1, '- Петькино -', 187000, 2, 1, 0.0, 293590, 'petkino')
Команда выполнена успешно
INSERT INTO channels VALUES (2, 'BARAKuda', 508000, 1, 2, 0.145833333333333, 797560, 'barakuda')
Команда выполнена успешно
INSERT INTO channels VALUES (3, 'varlamov', 196000000, 2, 3, 0.009523809523809525, 307720000, 'varlamov')
Команда выполнена успешно


0    None
1    None
2    None
dtype: object

In [38]:
bloggers = pd.read_excel('bloggers.xlsx')
bloggers['phone'] = bloggers['phone'].apply(lambda x: '+' + str(int(x)))
bloggers

,id,name,av_price,pricelist,phone
0,1,Дмитрий Львович Зильбертруд,12000,petkino,+79036110480
1,2,Борис Гонтермахер,57000,panorama,+79678934567
2,3,Илья Варламов,1000000,varlamov,+75498907892


In [39]:
cur.delete('bloggers')

DELETE FROM bloggers
Команда выполнена успешно


In [40]:
bloggers.apply(lambda x: cur.create('bloggers', values = list(x)), axis=1)

INSERT INTO bloggers VALUES (1, 'Дмитрий Львович Зильбертруд', 12000, 'petkino', '+79036110480')
Команда выполнена успешно
INSERT INTO bloggers VALUES (2, 'Борис Гонтермахер', 57000, 'panorama', '+79678934567')
Команда выполнена успешно
INSERT INTO bloggers VALUES (3, 'Илья Варламов', 1000000, 'varlamov', '+75498907892')
Команда выполнена успешно


0    None
1    None
2    None
dtype: object

In [41]:
print(cur.read('videos'))
print(cur.read('channels'))
print(cur.read('bloggers'))

SELECT * FROM videos
Команда выполнена успешно
[{'id_videos': 1, 'video_name': 'Заветный берег на озере Иткуль', 'channel_id': 1, 'views': 1530, 'date': datetime.date(2020, 10, 15), 'adv_a': 0, 'channel_id_videos': 1, 'blogger_id_videos': 1}, {'id_videos': 2, 'video_name': 'Путешествие в прошлое для МексЭтноЭксп', 'channel_id': 1, 'views': 7751, 'date': datetime.date(2019, 5, 20), 'adv_a': 0, 'channel_id_videos': 1, 'blogger_id_videos': 1}, {'id_videos': 3, 'video_name': 'Бакшевская масляница 2019', 'channel_id': 1, 'views': 12571, 'date': datetime.date(2019, 3, 15), 'adv_a': 0, 'channel_id_videos': 1, 'blogger_id_videos': 1}, {'id_videos': 4, 'video_name': 'Виталий Наливкин ловит наркоманов', 'channel_id': 2, 'views': 1284187, 'date': datetime.date(2020, 12, 13), 'adv_a': 0, 'channel_id_videos': 2, 'blogger_id_videos': 2}, {'id_videos': 5, 'video_name': 'Виталий Наливкин рушит бетонные плиты', 'channel_id': 2, 'views': 1707191, 'date': datetime.date(2020, 11, 22), 'adv_a': 1, 'channel

## SELECT с несколькими (от трех) таблицами в условии выборки

In [42]:
command = """
SELECT videos.video_name, channels.channel_name, videos.views, channels.subscribers, bloggers.av_price
FROM videos, channels, bloggers
WHERE videos.channel_id_videos = channels.id_channels
AND videos.blogger_id_videos = bloggers.id_bloggers
AND videos.adv_a = True 
AND channels.topic_id = 1
;
"""

In [43]:
cur.execute(command)
cur.cur.fetchall()


SELECT videos.video_name, channels.channel_name, videos.views, channels.subscribers, bloggers.av_price
FROM videos, channels, bloggers
WHERE videos.channel_id_videos = channels.id_channels
AND videos.blogger_id_videos = bloggers.id_bloggers
AND videos.adv_a = True 
AND channels.topic_id = 1
;

Команда выполнена успешно


[{'video_name': 'Виталий Наливкин рушит бетонные плиты',
  'channel_name': 'BARAKuda',
  'views': 1707191,
  'subscribers': 508000,
  'av_price': 57000.0},
 {'video_name': 'Виталий Наливкин Масочный режим',
  'channel_name': 'BARAKuda',
  'views': 1442579,
  'subscribers': 508000,
  'av_price': 57000.0},
 {'video_name': 'Виталий Наливкин спасает Камчатку',
  'channel_name': 'BARAKuda',
  'views': 1101153,
  'subscribers': 508000,
  'av_price': 57000.0}]

## Вложенные SELECT

In [44]:
# найдём каналы, которые ведут «дешевые» блогеры

In [45]:
command = """
SELECT channels.channel_name
FROM channels
WHERE blogger_id_channels IN (SELECT bloggers.id_bloggers 
                                FROM bloggers
                                WHERE bloggers.av_price < 50000)
;
"""

In [46]:
cur.execute(command)
cur.cur.fetchall()


SELECT channels.channel_name
FROM channels
WHERE blogger_id_channels IN (SELECT bloggers.id_bloggers 
                                FROM bloggers
                                WHERE bloggers.av_price < 50000)
;

Команда выполнена успешно


[{'channel_name': '- Петькино -'}]

## Запросы с JOIN

In [47]:
#склеим таблицы «видео» и «каналы» и покажем названия видео с каналов, у которых много подписчиков

In [48]:
command = """
SELECT * 
FROM videos v JOIN channels ch ON v.channel_id = ch.id_channels
WHERE ch.subscribers > 200000
;
"""

In [49]:
cur.execute(command)
cur.cur.fetchall()


SELECT * 
FROM videos v JOIN channels ch ON v.channel_id = ch.id_channels
WHERE ch.subscribers > 200000
;

Команда выполнена успешно


[{'id_videos': 4,
  'video_name': 'Виталий Наливкин ловит наркоманов',
  'channel_id': 2,
  'views': 797560,
  'date': datetime.date(2020, 12, 13),
  'adv_a': Decimal('0'),
  'channel_id_videos': 2,
  'blogger_id_videos': 2,
  'id_channels': 2,
  'channel_name': 'BARAKuda',
  'subscribers': 508000,
  'topic_id': 1,
  'blogger_id_channels': 2,
  'pickle': 'barakuda'},
 {'id_videos': 5,
  'video_name': 'Виталий Наливкин рушит бетонные плиты',
  'channel_id': 2,
  'views': 797560,
  'date': datetime.date(2020, 11, 22),
  'adv_a': Decimal('0'),
  'channel_id_videos': 2,
  'blogger_id_videos': 2,
  'id_channels': 2,
  'channel_name': 'BARAKuda',
  'subscribers': 508000,
  'topic_id': 1,
  'blogger_id_channels': 2,
  'pickle': 'barakuda'},
 {'id_videos': 6,
  'video_name': 'Виталий Наливкин Масочный режим',
  'channel_id': 2,
  'views': 797560,
  'date': datetime.date(2020, 11, 15),
  'adv_a': Decimal('0'),
  'channel_id_videos': 2,
  'blogger_id_videos': 2,
  'id_channels': 2,
  'channel_na